## Urban Heat Island (UHI) map for Washington DC
#In this tutorial, we attempt to quantify the temperature on July 8th, 2018 across Washington DC in the US.

## 1. Landsat (raster) temperature data for DC
#The input data used here is Landsat satellite image captured on the above-mentioned date.

#By running the following code, we can we get the summary of the Landsat Surface Temperature (LST) through raster functions:



In [ ]:
library(raster) # reading in and wrangling landsat data
library(sf) # reading in and wrangling contextual data
library(tidyverse)
library(NMF)
library(ggtext) # text in plots
library(showtext) # more fonts
library(tibble)
library(BiocManager)
library(ggplot2)
options(warn=-1)
font_add_google("Lato", regular.wt = 300, bold.wt = 700) #

#The image captured on 2018-07-08
landsat_dc_july18 <- raster("LST_F_20180708.tif") # saved the downloaded files in a data/ folder

#The image captured on 2015-07-16
#landsat_dc_july18 <- raster("LST_F_20180708.tif") # saved the downloaded files in a data/ folder

# water features in the city
water <- st_read("https://opendata.arcgis.com/datasets/db65ff0038ed4270acb1435d931201cf_24.geojson") %>%
  st_transform(st_crs(landsat_dc_july18)) # use the same coordinate reference system as the landsat data

landsat_dc_july18 # gives info like extent, crs, min/max


summary(landsat_dc_july18) # gives distribution summary based on a sample

#From this we can see that the surface temperature ranges from 72 to 106 F with a median of 88 F.

## 2. Minimum Viable Product map
#To plot the data, we can converte the raster values into a dataframe format. After that, using ggplot2 we can visualize the map.



In [ ]:
temp_spdf <- as(landsat_dc_july18, "SpatialPointsDataFrame") # create spatialpoints dataframe
temp_df <- as_tibble(temp_spdf) # convert that to a plain tibble
colnames(temp_df) <- c("value", "x", "y")

water_sp <- as(water, "Spatial")


ggplot() +
  geom_raster(data = temp_df, aes(x = x, y = y,  fill = value), interpolate = TRUE) +
  geom_polygon(data = water_sp, aes(x = long, y = lat, group = group), color = "gray90", fill = "white") +
  coord_equal() + # make sure that the x and y axis are comparable
  theme_minimal()


#As is shown, it will be hard to see the temperature patterns with this color scheme. A diverging color scheme would highlight relative low and high values. 

## 3. Adding A Color Scheme:

In [ ]:
ggplot() +
  geom_raster(data = temp_df, aes(x = x, y = y,  fill = value), interpolate = TRUE) +
  geom_polygon(data = water_sp, aes(x = long, y = lat, group = group), color = "gray90", fill = "white") +
  theme_minimal() +
  coord_equal() +
  scale_fill_gradientn(colors = rcartocolor::carto_pal(name = "TealRose", n = 7), breaks = c(72.5, 88, 106.6), labels = c("72", "88", "106"), name = "")

#Now, the temperature pattern is obvious across the city!.

## 4. Labeling the Map:





#We can also add the text neighborhood labels and descriptions in the margins.
#To do this we can use the ggtext package which allows for markdown formatting.
#To figure out the coordinates of the labels, we can use the axis ticks as a starting point and then try to find the right place. 

In [ ]:
labels <- tibble(x = c(396000, 395900, 390500,
                       399700, 396000, 399700, 399200, 389070, 390000),
                 y = c(138300, 136600, 140000,
                       133000, 140000, 140500, 146400, 147000, 130100),
                 lab = c("W A S H I N G T O N", "downtown", "palisades", "anacostia", "columbia<br> heights", "brookland", "**Hotter:** Hotspots in<br>Brookland, Columbia<br>Heights, and LeDroit<br>Park hit **100 to 106 F**", "**Cooler:** Forested Rock<br>Creek Park recorded the<br>city's lowest temperatures<br>and helped to cool down<br>surrounding areas", "**Urban green spaces**<br>are an invaluable<br>resources for cooling<br>urban neighborhoods.<br>They help promote<br>walkability and improve<br>quality of living. Even a<br>few trees help!"))
ggplot() +
  geom_raster(data = temp_df, aes(x = x, y = y,  fill = value), interpolate = TRUE) +
  geom_polygon(data = water_sp, aes(x = long, y = lat, group = group), color = "gray90", fill = "white") +
  geom_richtext(data = labels, aes(x = x, y = y, label = lab), fill = NA, label.color = NA, # remove background and outline
                label.padding = grid::unit(rep(0, 4), "pt"),  color = "#656c78", hjust = 0)+
  theme_minimal() +
  coord_equal() +
  scale_fill_gradientn(colors = rcartocolor::carto_pal(name = "TealRose", n = 7), breaks = c(72.5, 88, 106.6), labels = c("72", "88", "106"), name = "")



## 5. Reformatting the Legend

In [ ]:
ggplot() +
  geom_raster(data = temp_df, aes(x = x, y = y,  fill = value), interpolate = TRUE) +
  geom_polygon(data = water_sp, aes(x = long, y = lat, group = group), color = "gray90", fill = "white") +
  geom_richtext(data = labels, aes(x = x, y = y, label = lab), fill = NA, label.color = NA, # remove background and outline
                label.padding = grid::unit(rep(0, 4), "pt"),  color = "#656c78", hjust = 0)+
  theme_minimal() +
  coord_equal() +
  scale_fill_gradientn(colors = rcartocolor::carto_pal(name = "TealRose", n = 7), breaks = c(72.5, 88, 106.6), labels = c("72 F", "88", "106"), name = "") +
  theme(legend.position = "bottom",
        axis.text = element_blank(),
        axis.title = element_blank(),
        panel.grid = element_line("transparent"),
       # text = element_text(color = "#656c78", family = "Lato"),
        plot.caption = element_text(hjust = 0)) +
  guides(fill = guide_colourbar(barheight = 0.3, barwidth = 20, direction = "horizontal", ticks = FALSE)) +
  labs(caption = "July 8, 2018\nSource: DC Open Data")


## 6. Detect Hottest Area:

#To differentiate the hottest areas in the city, we can find places/buildings that their temperatures reach at least 95 F.



In [ ]:
options(warn=-1)
over95 <- temp_df %>%
  filter(value >=95) # where the value is 95 or greater

labels2 <- tibble(x = c(396000, 395900, 390500,
                        399700, 396000, 399700, 399200, 389000),
                  y = c(138300, 136600, 140000,
                        133000, 140000, 140500, 146400, 132500),
                  lab = c("W A S H I N G T O N", "downtown", "palisades", "anacostia", "columbia<br> heights", "brookland", "The locations in **red** all<br>reached at least **95 F** while<br> the median was only **88 F**", "The hotter areas are primarily<br>in the NE quadrant which has<br>historically been more industrial"))

ggplot() +
  geom_raster(data = temp_df, aes(x = x, y = y), fill = "gray90", interpolate = TRUE) +
  geom_raster(data = over95, aes(x = x, y = y), fill = "#d0587e") +
  geom_polygon(data = water_sp, aes(x = long, y = lat, group = group), color = "gray90", fill = "white") +
  geom_richtext(data = labels2, aes(x = x, y = y, label = lab), fill = NA, label.color = NA, # remove background and outline
                label.padding = grid::unit(rep(0, 4), "pt"),  color = "#656c78", hjust = 0)+
  theme_minimal() +
  coord_equal() +
  scale_fill_gradientn(colors = rcartocolor::carto_pal(name = "TealRose", n = 7), breaks = c(72.5, 88, 106.6), labels = c("72", "88", "106"), name = "") +
  theme(legend.position = "bottom",
        axis.text = element_blank(),
        axis.title = element_blank(),
        panel.grid = element_line("transparent"),
        #text = element_text(color = "#656c78", family = "Lato"),
        plot.caption = element_text(hjust = 0)) +
  guides(fill = guide_colourbar(barheight = 0.3, barwidth = 15, direction = "horizontal", ticks = FALSE)) +
  labs(caption = "July 8, 2018\nSource: DC Open Data")